In [19]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [21]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("ADK components imported successfully.")

ADK components imported successfully.


In [22]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]

    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
        </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("Helper functions defined.")

Helper functions defined.


In [23]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [32]:
#Research Agent : Its job is to use the google_search_tool and present findings.4
research_agent = Agent(
    name="Research_Agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print(" research_agent created.")

 research_agent created.


In [33]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="Summarizer_Agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("summarizer_agent created.")

summarizer_agent created.


In [34]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("root_agent created.")

root_agent created.


In [35]:
runner = InMemoryRunner(agent=root_agent)
print("Runner created.")

Runner created.


In [36]:
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > Here's a summary of the latest advancements in quantum computing and their implications for AI:

*   **Exponentially Enhanced Computational Power:** Quantum computers, using qubits and superposition, can process vast numbers of solutions simultaneously, dramatically speeding up AI training and complex problem-solving.
*   **Advanced AI Capabilities and Efficiency:** Quantum AI can uncover hidden patterns in data, improve areas like natural language processing, and may require fewer parameters, leading to more sustainable AI models.
*   **Overcoming Classical AI Limitations:** Quantum computing has the potential to tackle AI problems currently intractable for classical computers, accelerating breakthroughs in fields like drug discovery and materials science.
*   **Progress in Hardware, Algorithms, and Hybrid Approaches:** Ongoing development in stable qubits, error correction, and specialized quantum algorithms, along with the integration of quantum and classical s

In [51]:
# Outline Agent: Creates the initial blog post outline.
outline_Agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("outline_agent created.")

outline_agent created.


In [52]:
writer_Agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        api_key=GOOGLE_API_KEY,
        retry_options=retry_config
    ),
    instruction="""Write a detailed blog post based on the outline provided: {blog_outline} 
    Expand each section into 2-3 paragraphs. 
    Write in an engaging, conversational style.
    Use the outline structure but make it flow naturally.""",
    output_key="blog_draft",  # This goes to the editor
)

print("Writer agent created successfully!")

Writer agent created successfully!


In [53]:
editor_Agent = Agent(
    name="EditorAgent", 
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        api_key=GOOGLE_API_KEY,
        retry_options=retry_config
    ),
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog_post",  # Final output
)

print("Editor agent created successfully!")

Editor agent created successfully!


In [54]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_Agent, writer_Agent, editor_Agent],
)

print("Sequential Agent created.")

Sequential Agent created.


In [55]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ## Outline:

**Headline:** Supercharge Your Code: The Undeniable Benefits of Multi-Agent Systems for Developers

**Introduction Hook:** Tired of monolithic codebases that crumble under their own weight? Imagine a world where your software isn't a single, complex beast, but a coordinated team of intelligent, specialized agents working seamlessly together. That's the power of Multi-Agent Systems (MAS), and it's about to revolutionize how you build software.

**Main Sections:**

**1. Enhanced Modularity and Reusability: Building Blocks for Success**

*   **Decomposition Made Easy:** MAS naturally breaks down complex problems into smaller, manageable, and independent agents, each responsible for a specific task. This drastically reduces the cognitive load and makes your codebase far easier to understand and maintain.
*   **Plug-and-Play Comp

In [56]:
response = await runner.run_debug(
    "Write a blog post about sustainable software development practices"
)


 ### Continue session: debug_session_id

User > Write a blog post about sustainable software development practices
OutlineAgent > ## Outline:

**Headline:** Code Greener: Sustainable Software Development Practices for a Healthier Planet (and a Smarter Business)

**Introduction Hook:** In a world increasingly conscious of its environmental footprint, the digital realm is no exception. Software, often perceived as intangible, has a very real and growing impact on energy consumption and resource usage. But what if we told you that by adopting sustainable software development practices, you could not only reduce your environmental impact but also build more efficient, cost-effective, and future-proof applications?

**Main Sections:**

**1. The Environmental Footprint of Code: Beyond the Screen**

*   **Energy Consumption of Data Centers:** Discuss how the vast infrastructure supporting our digital world (servers, cooling systems, networks) consumes significant amounts of energy, contribut

In [57]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("tech_researcher created.")

tech_researcher created.


In [59]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print(" health_researcher created.")

 health_researcher created.


In [60]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("finance_researcher created.")

finance_researcher created.


In [61]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("aggregator_agent created.")

aggregator_agent created.


In [62]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("Parallel and Sequential Agents created.")

Parallel and Sequential Agents created.


In [63]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
HealthResearcher > ## Daily Executive Briefing: Tech, Health, and Finance

**Health:** Advancements in mRNA technology, pioneered by COVID-19 vaccines, are now being explored for numerous other diseases. Microneedle patches for vaccine delivery are nearing widespread adoption, potentially simplifying global immunization efforts. Within the next 5-10 years, AI is expected to significantly enhance diagnostic accuracy and treatment planning.

**Technology:** AI continues its rapid integration across sectors, with generative AI (GenAI) becoming more accessible for various business functions. Agentic AI, capable of autonomous planning and execution of tasks, is emerging as a key development. The expansion of 5G networks is enabling more sophisticated applications in areas like autonomous vehicles and smart cities, with widespread impact anticipated within the next decade.

**Fin

In [64]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [65]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [66]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


In [67]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [68]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [69]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias, a man whose life was as predictable as the tides, polished the great Fresnel lens. For thirty years, the rhythmic sweep of its beam had been his only companion. Tonight, however, was different. A storm raged, lashing the tower with sea spray, and a strange object bobbed against the rocks below.

He descended the winding stairs, lantern in hand. It was a sealed wooden box, barnacle-encrusted but strangely intact. Inside, nestled on velvet, lay a map unlike any he'd ever seen. Lines of phosphorescent ink pulsed with a soft, ethereal light, depicting islands that didn't exist on any charted course. A single, shimmering 'X' marked a spot in the unfathomable blue. Elias traced it with a calloused finger, a thrill of the unknown coursing through him, brighter than any lighthouse beam.
CriticAgent > The plot is intriguing, settin